In [28]:
import os
import time
import pickle
import itertools
from collections import namedtuple
import numpy as np
import matplotlib.pyplot as plt

from genEM3.data.wkwdata import WkwData,DataSource
from genEM3.util.path import get_data_dir
import genEM3.data.annotation as annotation 
from genEM3.data.annotation import Widget

In [29]:
# Loaded the json file for the dataset
json_dir = os.path.join(get_data_dir(), 'dense_3X_10_10_2_um/original_merged_without_myelin_v01.json') 
config = WkwData.config_wkwdata(json_dir)
dataset = WkwData.init_from_config(config)

# Get a set of data sources with the normal bounding boxes to create a patch wise detaset and a larger bounding box for annotation
margin = 35
roi_size = 140
source_dict = annotation.patch_source_list_from_dataset(dataset=dataset,
                                                        margin=margin,
                                                        roi_size=roi_size)
dataset_dict = dict.fromkeys(source_dict)

for key in source_dict:
    cur_source = source_dict[key]
    cur_patch_shape = tuple(cur_source[0].input_bbox[3:6])
    cur_config = WkwData.config_wkwdata(datasources_json_path=None,
                                        input_shape=cur_patch_shape,
                                        output_shape=cur_patch_shape)
    dataset_dict[key] = WkwData.init_from_config(cur_config, source_dict[key])
# assert larger and small datasets have the same length
dataset_lengths = [len(d) for d in dataset_dict.values()]
assert all(cur_L == dataset_lengths[0] for cur_L in dataset_lengths)
# break down the range into partitions of 1000
range_size = 1000
list_ranges = annotation.divide_range(total_size=len(dataset_dict['large']),
                                      chunk_size=range_size)

In [30]:
# Annotation
button_names = [['No', 'Yes'], ['No', 'Yes']]
target_classes = ['Myelin', 'Debris']
w = annotation.Widget(dataset=dataset_dict['large'], 
                    index_range=range(len(dataset_dict['large'])),
                    button_names = button_names,
                    target_classes = target_classes,
                    margin=margin, 
                    roi_size = roi_size)
w.show_widget()

In [32]:
filename = './test.pkl'
w.save(file_name=filename)
loaded_widget = Widget.load(file_name=filename)
print(loaded_widget.annotation_list[0:10])

[(0, {'Myelin': 1.0, 'Debris': 1.0}), (1, {'Myelin': 0.0, 'Debris': 1.0}), (2, {'Myelin': 1.0, 'Debris': 0.0}), (3, {'Myelin': 0.0, 'Debris': 1.0}), (4, {'Myelin': 1.0, 'Debris': 0.0}), (5, {'Myelin': 1.0, 'Debris': 1.0}), (6, {'Myelin': 1.0, 'Debris': 1.0}), (7, {'Myelin': 1.0, 'Debris': 1.0}), (8, {'Myelin': None, 'Debris': None}), (9, {'Myelin': None, 'Debris': None})]
